In [5]:
!pip install accelerate peft bitsandbytes transformers trl
!pip install pyarrow==14.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [6]:
from huggingface_hub import notebook_login
notebook_login()


In [7]:
#importing necessary packages
import torch
from datasets import load_dataset,Dataset
#importing lora config and corresponding peft package
from peft import LoraConfig,AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,TrainingArguments
from trl import SFTTrainer
import os
import accelerate

#Loading dataset from hugging face

### Transforming emotion dataset from integers to String classification

In [ ]:
emotion_dataset = load_dataset("dair-ai/emotion",trust_remote_code=True)

emotions=[]
classification={0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
# classification[emotion_dataset['train'][0]['label']] => 'sadness'

#merging all 20k into one list
for data in emotion_dataset['train']:
  dt=data.copy()
  dt['label'] = classification[dt['label']]
  emotions.append(dt)
for data in emotion_dataset['test']:
  dt=data.copy()
  dt['label'] = classification[dt['label']]
  emotions.append(dt)
for data in emotion_dataset['validation']:
  dt=data.copy()
  dt['label'] = classification[dt['label']]
  emotions.append(dt)

# transformed dataset to string representation
''' emotion_dataset with train\test\validation list all together'''
emotions

###Removing Unnamed data from dataset

In [ ]:
# elizabeth_novel_dataset = load_dataset("elizathornton/elizabeth_gaskell_unfinished_novel")
# elizabeth_novel=[]
# for data in elizabeth_novel_dataset['train']:
#   dt=data.copy()
#   del dt['Unnamed: 0']
#   elizabeth_novel.append(dt)
# # elizabeth_novel
# # elizabeth_novel_dataset['train'][1]

In [9]:
# dolphin_dataset = load_dataset("cognitivecomputations/dolphin") # subdataset of alpaca
# dolphin dataset contains instruction input output
# romance_novel_dataset = load_dataset("diltdicker/romance_novel_data-2022")
# poem_dataset = load_dataset("poem_sentiment")

# poem_train_dataset = load_dataset("othorizedshogun/poems_dataset")#have test dataset also

# text generation
# alpaca_dataset = load_dataset("tatsu-lab/alpaca")


# getting model and tokenizer
def model_and_tokenizer():
  # loading model from hugging face
  tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.4")
  tokenizer.pad_token=tokenizer.eos_token
  bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True)
  model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.4",device_map="auto")
  model.config.use_cache=False
  model.config.pretraining_tp=1
  return model,tokenizer

#output model name: De-Narration
output_model="DeNar_R-v1"

In [10]:
import pandas as pd
# converting the dataset into dataframe
emotion_data=pd.DataFrame(emotions)
'''First fine-tune dataset'''



# elizabeth_data=pd.DataFrame(elizabeth_novel)

'First fine-tune dataset'

In [ ]:
# the format of ChatLM (below)
def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

###Emotion dataset

In [11]:
emotion_label=emotion_data.copy()
emotion_text=emotion_data.copy()
emotion_text["train text"] =emotion_data[["label", "text"]].apply(lambda x: "<|im_start|>user\n " + x["label"] + " <|im_end|>\n<|im_start|>assistant\n " + x["text"] + " <|im_end|>\n", axis=1)
emotion_label["train text"] =emotion_data[["text", "label"]].apply(lambda x: "<|im_start|>user\n " + x["text"] + " <|im_end|>\n<|im_start|>assistant\n " + x["label"] + " <|im_end|>\n", axis=1)
emotion= pd.concat([emotion_text, emotion_label], ignore_index=True)
emotion
emotion_dataset=Dataset.from_pandas(emotion)

In [12]:
emotion_dataset["train text"][10343]

'<|im_start|>user\n joy <|im_end|>\n<|im_start|>assistant\n i feel fantastic now but am terrified of injuring my back again we provide our sizes for the following items <|im_end|>\n'

In [13]:
# getting model and tokenizer ready
model,tokenizer=model_and_tokenizer()

tokenizer_config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

In [14]:
# setting up LoRA
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [47]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-5,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=4,
        max_steps=300,
        fp16=True,
    )
trainer = SFTTrainer(
        model=model,
        train_dataset=emotion_dataset,# dataset for model
        peft_config=peft_config,
        dataset_text_field="train text", # using the text column of the data set that we formatted for our training purpose
        # it will be different for different dataset.
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [17]:
# merged LoRA with the base model


from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.4",torch_dtype=torch.float16, load_in_8bit=False,device_map="auto",trust_remote_code=True)
model_path = "/content/DeNar_R-v1/checkpoint-250"
peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")
model = peft_model.merge_and_unload()

#LLM

In [18]:
from transformers import GenerationConfig
from time import perf_counter

def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=12,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [43]:
generate_response(user_input='i love you')

user
i love you 
 assistant: i feel like a little girl in the first part of the
Time taken for inference: 0.34 seconds
